<a href="https://colab.research.google.com/github/SilasRu/Oeko3/blob/master/Copy_of_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SilasRu/Oeko3.git

Cloning into 'Oeko3'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (372/372), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 6587 (delta 16), reused 355 (delta 6), pack-reused 6215
Receiving objects: 100% (6587/6587), 472.96 MiB | 36.50 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (5154/5154), done.


**Importing packages and path handling**

In [0]:
import os
import numpy as np
from keras.preprocessing.image import  ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
import keras.models as km
import keras.layers as kl
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Dropout
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score

In [0]:
import sys
sys.path.append("[Oeko3\\data\\train\\spectrograms]")
path = os.path.join('Oeko3', 'data', 'train', 'spectrograms')
persons = os.listdir(path)

sys.path.append("[Oeko3\\data\\test\\spectrograms]")
path_test = os.path.join('Oeko3', 'data', 'test', 'spectrograms')
persons_test = os.listdir(path_test)


**Converting train and test spectrograms to matrices**

In [3]:
train_dic = {}
x_train = []
y_train = []
errors = 0
for i in range(len(persons)):
    tempdir = os.path.join(path, persons[i])
    tempfiles = os.listdir(tempdir)
    tempimglist = []
#     for j in range(len(tempfiles)):
    for j in range(90):
        temp_img = load_img(os.path.join(tempdir,tempfiles[j]))
        temp_x = img_to_array(temp_img)/255.
        tempimglist.append(temp_x)
        if i ==0 and j == 0:
            temp_x = temp_x.reshape((1,)+temp_x.shape)
            x_train = temp_x
            y_train.append(i)
        else:
            try:
                temp_x = temp_x.reshape((1,)+temp_x.shape)
                print(str(persons[i])+'__'+str(tempfiles[j])+', shape:  '+str(temp_x.shape))
                x_train = np.concatenate((x_train,temp_x),  axis = 0)
                y_train.append(i)
            except:
                errors += 1
                print(str(persons[i])+'__'+str(tempfiles[j]+'   INVALID SHAPE. CAN NOT ADD TO TRAINING SET'))
    train_dic[persons[i]] = np.array(tempimglist)

print(str(errors)+' ERRORS THAT CAN NOT BE ADDED TO TRAINING SET')

goess__Petra_G_6_064.png, shape:  (1, 480, 640, 3)
goess__Petra_G_6_075.png, shape:  (1, 480, 640, 3)
goess__Petra_G_3_30.png, shape:  (1, 480, 640, 3)
goess__Petra_G_2_54.png, shape:  (1, 480, 640, 3)
goess__Petra_G_3_11.png, shape:  (1, 480, 640, 3)
goess__Petra_G_3_75.png, shape:  (1, 480, 640, 3)
goess__Petra_G_2_10.png, shape:  (1, 288, 432, 3)
goess__Petra_G_2_10.png   INVALID SHAPE. CAN NOT ADD TO TRAINING SET
goess__Petra_G_4_17.png, shape:  (1, 480, 640, 3)
goess__Petra_G_6_100.png, shape:  (1, 480, 640, 3)
goess__Petra_G_4_24.png, shape:  (1, 480, 640, 3)
goess__Petra_G_6_047.png, shape:  (1, 480, 640, 3)
goess__Petra_G_4_29.png, shape:  (1, 480, 640, 3)
goess__Petra_G_2_04.png, shape:  (1, 288, 432, 3)
goess__Petra_G_2_04.png   INVALID SHAPE. CAN NOT ADD TO TRAINING SET
goess__Petra_G_4_43.png, shape:  (1, 480, 640, 3)
goess__Part1_0_42.png, shape:  (1, 288, 432, 3)
goess__Part1_0_42.png   INVALID SHAPE. CAN NOT ADD TO TRAINING SET
goess__Petra_G_3_83.png, shape:  (1, 480, 6

In [0]:
test_dic = {}
x_test = []
y_test = []
errors = 0
for i in range(len(persons_test)):
    tempdir = os.path.join(path_test, persons_test[i])
    tempfiles = os.listdir(tempdir)
    tempimglist = []
    for j in range(len(tempfiles)):
        temp_img = load_img(os.path.join(tempdir,tempfiles[j]))
        temp_x = img_to_array(temp_img)/255.
        tempimglist.append(temp_x)
        if i ==0 and j == 0:
            temp_x = temp_x.reshape((1,)+temp_x.shape)
            x_test = temp_x
            y_test.append(i)
        else:
            try:
                temp_x = temp_x.reshape((1,)+temp_x.shape)
                print(str(persons[i])+'__'+str(tempfiles[j])+', shape:  '+str(temp_x.shape))
                x_test = np.concatenate((x_test,temp_x),  axis = 0)
                y_test.append(i)
            except:
                errors += 1
                print(str(persons_test[i])+'__'+str(tempfiles[j]+'   INVALID SHAPE. CAN NOT ADD TO TRAINING SET'))
    train_dic[persons_test[i]] = np.array(tempimglist)

print(str(errors)+' ERRORS THAT CAN NOT BE ADDED TO TRAINING SET')

**CNN build**

In [4]:
X_train = x_train
y_lab = np.array(y_train)

y_train = to_categorical(y_train)

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64,strides=3, kernel_size=3, activation="relu",
                 input_shape= X_train[0].shape))
model.add(MaxPooling2D())
model.add(Conv2D(32,strides=3 ,kernel_size=3, activation="sigmoid"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(len(persons), activation="softmax"))

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


**Fitting the model on the training set**

In [0]:
model.fit(X_train, y_train, validation_split = 0.2, epochs=20, shuffle=True)

**Prediction on the test set and confusion matrix**

In [54]:
prediction = model.predict_classes(x_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))

[[30  0  2  0]
 [ 0 48  2  0]
 [ 2  1 47  0]
 [ 0  0  0 50]]
0.9615384615384616
